# Imports

In [94]:
import numpy as np
import matplotlib.pyplot as plt
import math 
from bresenham import bresenham
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.filters import median
from datetime import datetime 
import pydicom
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import ExplicitVRLittleEndian
from pydicom._storage_sopclass_uids import MRImageStorage
import ipywidgets as widgets
from dateutil.parser import parse
import re

# Global variables

In [95]:
STEP = 1 # step of emitter in degrees
NUMBER_OF_DETECTORS = 90
DETECTORS_RANGE = 90 # degrees
RANGE = 360 # iterations
RADIUS = int() # initialized and compute after image is loaded
IS_FILTER = True
IS_DICOM = True
SOURCE = "photos/Shepp_logan.jpg"

PATIENT_NAME = "Karol Okrasa"
DESCRIPTION = "description"
DATE = datetime.now()

original = list()
input_image = list()
reverses = list()
sinograms = list()

# Main

In [96]:
def main():
    image = rgb2gray(imread(SOURCE))
    global original
    original = np.copy(image)
    resized = resize(image, (len(image) / 4, len(image[0]) / 4))
    new_edge = round(math.sqrt(2) * max(len(resized), len(resized[0])))
    radius = round(new_edge / 2)

    image = np.zeros([new_edge, new_edge])
    image[radius - round(len(resized) / 2) : radius + round(len(resized) / 2), radius - round(len(resized[0]) / 2) : radius + round(len(resized[0]) / 2)] = resized
    
    global input_image
    global RADIUS     
    input_image = image
    RADIUS = round(len(image) / 2) - 1

    
    sinogram = make_sinogram(image)
    
    reverse_sinogram = make_reverse_sinogram(sinogram, [len(image), len(image[0])])
    reverse_sinogram = reverse_sinogram[round(len(image) / 4) : 3 * round(len(image) / 4), round(len(image[0]) / 4) : 3 * round(len(image[0]) / 4)]
    reverse_sinogram = normalize(reverse_sinogram)

    if IS_FILTER:
        reverse_sinogram = median(reverse_sinogram, selem=np.ones((5, 5)))

    if IS_DICOM:
        create_dicom(normalized_reverse_sinogram)
        dicom_image = pydicom.dcmread("output.dcm")

    image = original[round(len(image) / 4) : 3 * round(len(image) / 4), round(len(image[0]) / 4) : 3 * round(len(image[0]) / 4)]
    display_images(image, sinogram, reverse_sinogram)

# Functions

### Display Images

In [97]:
def display_images(image, sinogram, reverse):
    tmp_image = original[round(len(image) / 4) : 3 * round(len(image) / 4), round(len(image[0]) / 4) : 3 * round(len(image[0]) / 4)]
    print("Błąd średiokwadratowy: " + str(mse(tmp_image, reverse)))
    
    plt.close()
    
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
    axes[0].set_title('Obraz wejściowy')
    axes[1].set_title('Sinogram')
    axes[2].set_title('Obraz wyjściowy')
    
    axes[0].imshow(image, cmap='gray')
    axes[1].imshow(sinogram, cmap='gray')
    axes[2].imshow(reverse, cmap='gray')
    
    axes[0].axis('off')
    axes[1].axis('off')
    axes[2].axis('off')
    
    plt.show()

### Emiter position

In [98]:
def set_emitter_position(angle):
    x = RADIUS * math.cos(math.radians(angle)) + RADIUS
    y = RADIUS * math.sin(math.radians(angle)) + RADIUS
    return [round(x), round(y)]

### Detectors positions

In [99]:
def set_detectors_positions(angle):
    result = list()
    
    for i in range(NUMBER_OF_DETECTORS):
        x = RADIUS * math.cos(math.radians(angle) + math.pi - (math.radians(DETECTORS_RANGE) / 2) 
                         + i * (math.radians(DETECTORS_RANGE) / (NUMBER_OF_DETECTORS - 1)))
        y = RADIUS * math.sin(math.radians(angle) + math.pi - (math.radians(DETECTORS_RANGE) / 2) 
                         + i * (math.radians(DETECTORS_RANGE) / (NUMBER_OF_DETECTORS - 1)))
        result.append([round(x + RADIUS), round(y + RADIUS)])
    
    return result

### All positions

In [100]:
def set_positions():
    emitters = list()
    detectors = list()
    
    for i in range (0, RANGE, STEP):
        emitters.append(set_emitter_position(i))
        detectors.append(set_detectors_positions(i))
        
    return emitters, detectors

### Sinogram

In [101]:
def make_sinogram(image):
    emitters, detectors = set_positions()
    sinogram = np.zeros([len(emitters), len(detectors[0])])
    
    for i, [emitterX, emitterY] in enumerate(emitters):
        for j, [detectorX, detectorY] in enumerate(detectors[i]):
            line = bresenham(emitterX, emitterY, detectorX, detectorY)

            for x, y in line:
                sinogram[i, j] += image[x, y]
                
        sinograms.append(normalize(sinogram))
        
    return sinogram 

### Reverse Sinogram

In [102]:
def make_reverse_sinogram(image, dimensions):
    emitters, detectors = set_positions()
    reverse_sinogram = np.zeros(dimensions)
    
    for i, [emitterX, emitterY] in enumerate(emitters):
        for j, [detectorX, detectorY] in enumerate(detectors[i]):
            line = bresenham(emitterX, emitterY, detectorX, detectorY)
            
            for x, y in line:
                reverse_sinogram[x, y] += image[i][j]
                
        reverse = reverse_sinogram[round(len(image) / 4) : 3 * round(len(image) / 4), round(len(image[0]) / 4) : 3 * round(len(image[0]) / 4)]
        reverse = normalize(reverse)
        reverses.append(reverse)
    
    return reverse

### Normalize

In [103]:
def normalize(image):
    min_value = np.min(image)
    max_value = np.max(image)

    for i in range(len(image)):
        for j in range(len(image[i])):
            image[i, j] = (image[i, j] - min_value) / (max_value - min_value) 

    return image

### MSE

In [104]:
def mse(x, y):
    err = np.sum((x.astype("float") - y.astype("float")) ** 2)
    err /= float(len(x) * len(x[0]))
    return err

# DICOM

In [105]:
def create_dicom(image):
    image *= 256
    image = image.astype(np.uint16)

    file_meta = Dataset()
    file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'
    file_meta.MediaStorageSOPInstanceUID = "1.2.3"
    file_meta.ImplementationClassUID = "1.2.3.4"
    file_meta.TransferSyntaxUID = ExplicitVRLittleEndian

    ds = FileDataset("output.dcm", {}, file_meta=file_meta, preamble=b"\0" * 128)
    ds.is_little_endian = True
    ds.is_implicit_VR = False
    ds.PatientName = PATIENT_NAME
    ds.ContentDate = DATE.strftime('%Y%m%d')
    ds.ContentTime = DATE.strftime('%H%M%S.%f')
    ds.StudyDescription = DESCRIPTION
    ds.is_little_endian = True
    ds.is_implicit_VR = False

    ds.PixelRepresentation = 1
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.HighBit = 15
    ds.BitsStored = 16
    ds.BitsAllocated = 16
    ds.SmallestImagePixelValue = str.encode('\x00\x00')
    ds.LargestImagePixelValue = str.encode('\xff\xff')
    ds.SOPClassUID = MRImageStorage
    ds.Columns = len(image)
    ds.Rows = len(image[0])

    ds.PixelData = image.tobytes()

    ds.save_as("output.dcm")

# GUI

In [106]:
#Sliders
sinogram_iter = widgets.IntSlider(description = "Sinogram Iteration", min=1, max=360, step=1, value=360) # iterations 
output_iter = widgets.IntSlider(description = "Output Iteration", min=1, max=360, step=1, value=360) # iterations 
step_slider = widgets.IntSlider(description = "Step in degrees", min=1, max=36, step=1, value=1) # step of emitter in degrees
detectors_slider = widgets.IntSlider(description = "Number of Detectors", min=10, max=300, step=5, value=90)
detector_range_slider = widgets.IntSlider(description = "Detectors range", min=0, max=360, step=10, value=90) # degrees
sliders = [sinogram_iter, output_iter, step_slider, detectors_slider, detector_range_slider]

def on_change_sinogram(v):
    display_images(input_image, sinograms[v.new - 1], reverses[output_iter.value - 1])
    
def on_change_output(v):
    reverse = reverses[v.new - 1]
    if IS_FILTER:
        reverse = median(reverse, selem=np.ones((5, 5))) 
    display_images(input_image, sinograms[sinogram_iter.value - 1], reverse)
    
def on_change_STEP(v):
    global STEP
    STEP = v.new
    
def on_change_NUMBER_OF_DETECTORS(v):
    global NUMBER_OF_DETECTORS
    NUMBER_OF_DETECTORS = v.new
    
def on_change_DETECTORS_RANGE(v):
    global DETECTORS_RANGE
    DETECTORS_RANGE = v.new

sinogram_iter.observe(on_change_sinogram, names="value")
output_iter.observe(on_change_output, names="value")
step_slider.observe(on_change_STEP, names="value")
detectors_slider.observe(on_change_NUMBER_OF_DETECTORS, names="value")
detector_range_slider.observe(on_change_DETECTORS_RANGE, names="value")

for slider in sliders:
    hbox=widgets.HBox([widgets.Label(slider.description),slider])
    slider.description=""
    display(hbox)
    

#TEXT FIELDS
texts = []    
texts.append(widgets.Text(description="Nazwa Pacjenta",value=PATIENT_NAME))
texts.append(widgets.Text(description="Opis pacjenta", value=DESCRIPTION))
texts.append(widgets.Text(description="Data badania", placeholder='DD-MM-YYYY', value=DATE.strftime("%d-%m-%Y")))

for text in texts:
    hbox=widgets.HBox([widgets.Label(text.description),text])
    text.description=""
    display(hbox)

def updateName(text):
    PATIENT_NAME = texts[0].value
def updateDescription(text):
    DESCRIPTION = texts[1].value

def updateDate(text):
    global DATE
    c1 = re.search("^[0-2][0-8]-[0][2]-[1-2][0-9][0-9][0-9]$", texts[2].value) #February
    c1a = re.search("^[2][9]-[0][2]-[1-2][0-9][0-9][0,2,4,6,8]$", texts[2].value) #February
    c1b = re.search("^[2][9]-[0][2]-[1-2][0-9][0-9][0,2,4,6,8]$", texts[2].value) #February
    c2 = re.search("^[0-2][0-9]-[0][4,6,9]-[1-2][0-9][0-9][0-9]$", texts[2].value) #Short months 
    c3 = re.search("^[3][0]-[0][4,6,9]-[1-2][0-9][0-9][0-9]$", texts[2].value) #Short months 
    c4 = re.search("^[0-2][0-9]-[1][1]-[1-2][0-9][0-9][0-9]$", texts[2].value) #Short months
    c5 = re.search("^[3][0]-[1][1]-[1-2][0-9][0-9][0-9]$", texts[2].value) #Short months
    
    c6 = re.search("^[0-2][0-9]-[0][1,3,5,7,8]-[1-2][0-9][0-9][0-9]$", texts[2].value) #Long months
    c7 = re.search("^[0-2][0-9]-[0][1,3,5,7,8]-[1-2][0-9][0-9][0-9]$", texts[2].value) #Long months
    c8 = re.search("^[3][0-1]-[1][0,2]-[1-2][0-9][0-9][0-9]$", texts[2].value) #Long months
    c9 = re.search("^[3][0-1]-[1][0,2]-[1-2][0-9][0-9][0-9]$", texts[2].value) #Long months
    if (c1 or c2 or c3 or c4 or c5 or c6 or c7 or c8 or c9):
        DATE = parse(texts[2].value)
    print(DATE.date())


texts[0].observe(updateName)
texts[1].observe(updateDescription)
texts[2].observe(updateDate)

#CheckBoxes
checkboxes = []
checkboxes.append(widgets.Checkbox(description="Filtering", value=True))
checkboxes.append(widgets.Checkbox(description="DICOM", value=True))
box=widgets.HBox([checkboxes[0],checkboxes[1]])
display(box)

def isFilteringUpdate(cb):
    global IS_FILTER
    global IS_DICOM
    IS_FILTER = checkboxes[0].value
    if (checkboxes[0].value==False):
        IS_DICOM = False
        checkboxes[1].value = False
        
def isDicomUpdate(cb):
    global IS_FILTER
    global IS_DICOM
    IS_DICOM = checkboxes[1].value
    if (checkboxes[1].value==True):
        IS_FILTER = True
        checkboxes[0].value = True
    
checkboxes[0].observe(isFilteringUpdate)
checkboxes[1].observe(isDicomUpdate)

#Start Button
startButton = widgets.Button(description="START", button_style="danger")
display(startButton)

def start(b):
    main()

startButton.on_click(start)

Button(button_style='danger', description='START', style=ButtonStyle())

c:\users\mikol\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


ValueError: zero-size array to reduction operation minimum which has no identity